In [1]:
# Handle different types of transformation
# different nb of pixels
# different nb of level of gray

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import tree
import os
import random

mpl.rc('image', cmap='gray_r')
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from keras.datasets import mnist

C:\Users\MariO\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Useful methods

In [4]:
# Transformation matrices
def create_PPstar(transform_type, nbpix):
    if transform_type is "translation":
        # nb of pixels translated
        nbt = int(np.round(0.25*nbpix))

        # bottom left square block
        bl = np.eye(nbpix-nbt,nbpix-nbt)
        # upper right square block
        ur = np.eye(nbt,nbt)
        # upper left rect block
        ul = np.zeros((nbt,nbpix-nbt))
        # bottom right rect block
        br = np.zeros((nbpix-nbt,nbt))

        # concatenate the blocks to build the transformation matrix
        P = np.concatenate((np.concatenate((ul,ur), axis=1), np.concatenate((bl,br), axis=1)), axis=0)
        Pstar = np.linalg.inv(P)
    return (P,Pstar)

In [5]:
# plt.figure(1)
def remove_ticks():
    plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
    plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    left=False,
    labelleft=False,
    labelbottom=False) # labels along the bottom edge are off    
    return

def vis_matrices(Mr, Mm, Mst, MstM):
#     print(Mr.shape[0])
    if Mr.shape[0]==1:
        Mr = Mr.reshape(int(np.sqrt(Mr.shape[1])), int(np.sqrt(Mr.shape[1])))
        Mm = Mm.reshape(int(np.sqrt(Mm.shape[1])), int(np.sqrt(Mm.shape[1])))
        Mst = Mst.reshape(int(np.sqrt(Mst.shape[1])), int(np.sqrt(Mst.shape[1])))
        MstM = MstM.reshape(int(np.sqrt(Mst.shape[1])), int(np.sqrt(Mst.shape[1])))

    
    fig, (ax1,ax2) = plt.subplots(nrows=2,ncols=2) #, squeeze=True, sharey=True)
    fig.set_size_inches(8,8)

    plt.sca(ax1[0])
    plt.imshow(Mr)
    plt.title("Real image")
    remove_ticks()

    plt.sca(ax2[0])
    plt.imshow(Mm)
    plt.title("Distorted image")
    remove_ticks()

    plt.sca(ax1[1])
    plt.imshow(Mst)
    plt.title("Corrected input")
    remove_ticks()

    plt.sca(ax2[1])
    plt.imshow(MstM)
    plt.title("Corrected visualization")
    remove_ticks()

In [6]:
def create_training_data(nbpix,nb_lvl_gray=1):
    """
    Build the set of one pixel matrices and apply the transformations
    """
    # List of indexes where to put a black pixel
    idx = range(0,nbpix**2,1)

    # Initialize empty arrays 
    Mreal_tab = np.zeros((len(idx),nbpix*nbpix))
    Mmang_tab = np.zeros((len(idx),nbpix*nbpix))
    Mstar_tab = np.zeros((len(idx),nbpix*nbpix))
    for ii in idx:
        # Fill the indexed pixel with a one
        Mreal_tab[ii,ii] = 1
        # Use the transformation matrices to generate Mmang and Mstar
        Mmang_tab[ii,:] = (Mreal_tab[ii,:].reshape(nbpix,nbpix)@P).reshape(1,nbpix**2)
        Mstar_tab[ii,:] = (Mreal_tab[ii,:].reshape(nbpix,nbpix)@Pstar).reshape(1,nbpix**2)
    return (Mreal_tab, Mmang_tab, Mstar_tab)

In [7]:
# Function to add one on the designated index
def add_one(arr,index):
    arr[0,index] = 1
    return arr

# Function to split the matrix into matrices of one pixel
def split_matrix(mat):
    """ Split a matrix into matrices of one pixel"""
    # find the indexes of the ones values in the matrix
    idx_arr = np.where(mat.reshape(1,nbpix**2)[0]==1)[0]
    # transform from an array to a list
    idx_lst = [idx_arr.item(ii) for ii in range(0,len(idx_arr))] 
    temp = np.zeros((1,nbpix**2))
    mat_split = np.zeros((len(idx_lst), nbpix**2))
#     Mtest = np.zeros((nbpix,nbpix))
    for ii in range(0,len(idx_lst)):
        mat_split[ii,idx_lst[ii]] = 1    
#         Mtest = Mtest + Mreal_split[ii,:].reshape(nbpix,nbpix)
    return mat_split

In [8]:
# Prediction functions
def predict_mangler(mat):
    # split the input matrix into matrices of one pixel
    mat_split = split_matrix(mat)
    nb_dark_pxl = mat_split.shape[0]
    
    # initializations
    res = np.zeros((nb_dark_pxl, nbpix**2))
    Mres = np.zeros((nbpix,nbpix))
    # Loop on all the one pixels array
    for ii in range(0,nb_dark_pxl):
        # Apply the models to the one pixel matrices
        res[ii,:] = model_mangler.predict(np.expand_dims(mat_split[ii], axis=0))
        # Sum up the results to construct the matrices
        Mres = Mres + res[ii,:].reshape(nbpix,nbpix)
    return Mres

def predict_corrector(mat):
    # split the input matrix into matrices of one pixel
    mat_split = split_matrix(mat)
    nb_dark_pxl = mat_split.shape[0]
    
    # initializations
    res = np.zeros((nb_dark_pxl, nbpix**2))
    Mres = np.zeros((nbpix,nbpix))
    # Loop on all the one pixels array
    for ii in range(0,nb_dark_pxl):
        # Apply the models to the one pixel matrices
        res[ii,:] = model_corrector.predict(np.expand_dims(mat_split[ii], axis=0))
        # Sum up the results to construct the matrices
        Mres = Mres + res[ii,:].reshape(nbpix,nbpix)
    return Mres

 ## MAIN

In [28]:
# Parameters
nbpix = 6 # nb of pixel to consider (nbpix x nbpix)
transform_type = "translation" # type of transformation
nb_lvl_gray = 1 # nb of level of gray
# Mreal = 

nbborder = int(np.round(nbpix*0.3))
nbborder
u = np.ones((nbborder,nbpix))
l = np.ones((nbpix-nbborder,nbborder))
r = l
door = np.zeros((nbpix-nbborder, nbpix-nbborder))
door

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [29]:
# create the transformation matrices
(P,Pstar) = create_PPstar("translation",nbpix)

# Create the training data
(Mreal_tab, Mmang_tab, Mstar_tab) = create_training_data(nbpix,1)
idx = range(0,nbpix**2,1)

# the training input are the matrices of the real images with one pixel
X_train = Mreal_tab

# the training output will be the category corresponding to the position of the pixel in the mangled image
y_train = [np.where(Mmang_tab[ii,:]==1)[0].item(0) for ii in idx]
y_train = to_categorical(y_train, nbpix**2)

# or the position of the pixel in the corrected image
y_train_cor = [np.where(Mstar_tab[ii,:]==1)[0].item(0) for ii in idx]
y_train_cor = to_categorical(y_train_cor, nbpix**2)

# creating the models
model_mangler = Sequential()
model_mangler.add(Dense(100, activation='relu', input_dim=nbpix**2))
model_mangler.add(Dense(100, activation='relu'))
model_mangler.add(Dense(nbpix**2, activation='softmax'))
# Inversion strategy doesnt work very well...
# we are going to try using another model, trained on the corrected input.
model_corrector = Sequential()
model_corrector.add(Dense(100, activation='relu', input_dim=nbpix**2))
model_corrector.add(Dense(100, activation='relu'))
model_corrector.add(Dense(nbpix**2, activation='softmax'))

# Compile the models
model_mangler.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])
model_corrector.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

# train the model "mangler"
model_mangler.fit( X_train,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

# train the corrected input model
model_corrector.fit( X_train,
    y_train_cor,
    epochs=100,
    shuffle=True,
    verbose=2
)    

Epoch 1/100
 - 0s - loss: 3.6019 - acc: 0.0278
Epoch 2/100
 - 0s - loss: 3.5758 - acc: 0.0556
Epoch 3/100
 - 0s - loss: 3.5567 - acc: 0.0556
Epoch 4/100
 - 0s - loss: 3.5393 - acc: 0.0833
Epoch 5/100
 - 0s - loss: 3.5221 - acc: 0.1111
Epoch 6/100
 - 0s - loss: 3.5057 - acc: 0.1667
Epoch 7/100
 - 0s - loss: 3.4902 - acc: 0.1944
Epoch 8/100
 - 0s - loss: 3.4742 - acc: 0.3056
Epoch 9/100
 - 0s - loss: 3.4582 - acc: 0.3333
Epoch 10/100
 - 0s - loss: 3.4425 - acc: 0.3611
Epoch 11/100
 - 0s - loss: 3.4268 - acc: 0.4167
Epoch 12/100
 - 0s - loss: 3.4102 - acc: 0.4167
Epoch 13/100
 - 0s - loss: 3.3940 - acc: 0.4167
Epoch 14/100
 - 0s - loss: 3.3771 - acc: 0.4722
Epoch 15/100
 - 0s - loss: 3.3606 - acc: 0.5833
Epoch 16/100
 - 0s - loss: 3.3431 - acc: 0.6667
Epoch 17/100
 - 0s - loss: 3.3256 - acc: 0.7222
Epoch 18/100
 - 0s - loss: 3.3076 - acc: 0.7500
Epoch 19/100
 - 0s - loss: 3.2895 - acc: 0.7778
Epoch 20/100
 - 0s - loss: 3.2704 - acc: 0.8333
Epoch 21/100
 - 0s - loss: 3.2499 - acc: 0.8611
E

Epoch 72/100
 - 0s - loss: 1.0512 - acc: 1.0000
Epoch 73/100
 - 0s - loss: 0.9992 - acc: 1.0000
Epoch 74/100
 - 0s - loss: 0.9483 - acc: 1.0000
Epoch 75/100
 - 0s - loss: 0.9002 - acc: 1.0000
Epoch 76/100
 - 0s - loss: 0.8543 - acc: 1.0000
Epoch 77/100
 - 0s - loss: 0.8074 - acc: 1.0000
Epoch 78/100
 - 0s - loss: 0.7652 - acc: 1.0000
Epoch 79/100
 - 0s - loss: 0.7224 - acc: 1.0000
Epoch 80/100
 - 0s - loss: 0.6833 - acc: 1.0000
Epoch 81/100
 - 0s - loss: 0.6470 - acc: 1.0000
Epoch 82/100
 - 0s - loss: 0.6100 - acc: 1.0000
Epoch 83/100
 - 0s - loss: 0.5760 - acc: 1.0000
Epoch 84/100
 - 0s - loss: 0.5425 - acc: 1.0000
Epoch 85/100
 - 0s - loss: 0.5114 - acc: 1.0000
Epoch 86/100
 - 0s - loss: 0.4840 - acc: 1.0000
Epoch 87/100
 - 0s - loss: 0.4570 - acc: 1.0000
Epoch 88/100
 - 0s - loss: 0.4317 - acc: 1.0000
Epoch 89/100
 - 0s - loss: 0.4088 - acc: 1.0000
Epoch 90/100
 - 0s - loss: 0.3870 - acc: 1.0000
Epoch 91/100
 - 0s - loss: 0.3663 - acc: 1.0000
Epoch 92/100
 - 0s - loss: 0.3457 - acc: